In [1]:
import json
import pandas as pd
import os
from collections import defaultdict

In [2]:
with open('../config.json') as config_file:
    config = json.load(config_file)
old_data_path = config['old_data_path']

print(old_data_path)

/scratch2/hle


In [3]:
# result data from fine tuned detectron
new_data_path = '/scratch2/thdaryan/data/fine_tuned'

In [4]:
df_old = pd.read_csv(os.path.join(old_data_path,f'refCOCO/test/attr_tables_with_target_box/attr_0.tsv'), encoding='utf-8',sep='\t')
df_old.head()

,box_alias,image_id,ann_id,ref_id,salience,x1,y1,w,h,TYPE_yolk,...,ATTR_thin,ATTR_decorative,ATTR_wrinkled,ATTR_peeling,ATTR_golden,ATTR_metallic,ATTR_back,ATTR_thick,ATTR_black and white,ATTR_leaning
0,trunk-1,296747,469306,24882,0.058165,411.079834,256.495056,85.829346,199.942230,0.000624,...,0.002494,0.002491,0.002523,0.002491,0.002491,0.002491,0.002500,0.002497,0.002491,0.002491
1,man-1,296747,469306,24882,0.246415,160.744263,125.726784,216.845001,335.273224,0.000624,...,0.002495,0.002494,0.002497,0.002494,0.002494,0.002494,0.002496,0.002494,0.002495,0.002518
2,elephant-1,296747,469306,24882,0.298262,434.499969,19.280037,205.287811,428.662079,0.000624,...,0.002494,0.002493,0.002627,0.002493,0.002493,0.002493,0.002496,0.002498,0.002493,0.002494
3,hair-1,296747,469306,24882,0.021163,186.153610,83.822449,77.811691,80.242737,0.000624,...,0.002494,0.002493,0.002494,0.002493,0.002493,0.002493,0.002495,0.002500,0.002496,0.002493
4,shirt-1,296747,469306,24882,0.108199,124.160843,161.846359,173.210648,184.301651,0.000624,...,0.002492,0.002492,0.002503,0.002492,0.002492,0.002492,0.002493,0.002495,0.002492,0.002492


In [5]:
df_new= pd.read_csv(os.path.join(new_data_path,f'refCOCO/test/attr_tables_with_target_box/attr_0.tsv'), encoding='utf-8',sep='\t')
df_new.head()

,box_alias,image_id,ann_id,ref_id,salience,x1,y1,w,h,TYPE_None,...,ATTR_thin,ATTR_decorative,ATTR_wrinkled,ATTR_peeling,ATTR_golden,ATTR_metallic,ATTR_back,ATTR_thick,ATTR_black and white,ATTR_leaning
0,person-1,296747,469306,24882,0.442801,71.536621,88.425377,366.361572,356.598450,0.010673,...,0.002500,0.002496,0.00250,0.002496,0.002496,0.002497,0.002501,0.002498,0.002501,0.002517
1,elephant-1,296747,469306,24882,0.369482,387.983887,0.000000,252.016113,432.559174,0.010681,...,0.002501,0.002497,0.00258,0.002497,0.002497,0.002497,0.002507,0.002510,0.002497,0.002502


In [6]:
df_new['TYPE_OOV']

0    0.010797
1    0.011065
Name: TYPE_OOV, dtype: float64

In [7]:
# drop type none in new data
list_col = df_new.columns
for col in list_col:
    if(len(col) >= len('TYPE_None') and col[:len('TYPE_None')]=='TYPE_None'):
        df_new.drop([col], axis=1, inplace = True)

In [8]:
list_type_old_data = [col for col in df_old.columns if 'TYPE_' in col]
len(list_type_old_data)

1601

In [9]:
list_type_new_data = [col for col in df_new.columns if 'TYPE_' in col]
len(list_type_new_data)

81

### Make the columns from old df and new df same

In [10]:
# the merged columns will be => type_old_data | type_new_data | attributes

In [11]:
for col in list_type_old_data:
    if (col not in list_type_new_data):
        df_new[col] = 0
        
for col in list_type_new_data:
    if (col not in list_type_old_data):
        df_old[col] = 0

In [12]:
df_new

,box_alias,image_id,ann_id,ref_id,salience,x1,y1,w,h,TYPE_person,...,TYPE_microphone,TYPE_blades,TYPE_towel_rack,TYPE_coaster,TYPE_star,TYPE_petals,TYPE_text,TYPE_feather,TYPE_spots,TYPE_buoy
0,person-1,296747,469306,24882,0.442801,71.536621,88.425377,366.361572,356.598450,0.028618,...,0,0,0,0,0,0,0,0,0,0
1,elephant-1,296747,469306,24882,0.369482,387.983887,0.000000,252.016113,432.559174,0.010734,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df_old

,box_alias,image_id,ann_id,ref_id,salience,x1,y1,w,h,TYPE_yolk,...,TYPE_baseball glove,TYPE_tennis racket,TYPE_wine glass,TYPE_hot dog,TYPE_potted plant,TYPE_dining table,TYPE_tv,TYPE_cell phone,TYPE_teddy bear,TYPE_hair drier
0,trunk-1,296747,469306,24882,0.058165,411.079834,256.495056,85.829346,199.942230,0.000624,...,0,0,0,0,0,0,0,0,0,0
1,man-1,296747,469306,24882,0.246415,160.744263,125.726784,216.845001,335.273224,0.000624,...,0,0,0,0,0,0,0,0,0,0
2,elephant-1,296747,469306,24882,0.298262,434.499969,19.280037,205.287811,428.662079,0.000624,...,0,0,0,0,0,0,0,0,0,0
3,hair-1,296747,469306,24882,0.021163,186.153610,83.822449,77.811691,80.242737,0.000624,...,0,0,0,0,0,0,0,0,0,0
4,shirt-1,296747,469306,24882,0.108199,124.160843,161.846359,173.210648,184.301651,0.000624,...,0,0,0,0,0,0,0,0,0,0
5,head-1,296747,469306,24882,0.182997,409.485199,0.000000,229.364655,235.395401,0.000624,...,0,0,0,0,0,0,0,0,0,0
6,face-1,296747,469306,24882,0.162378,455.965698,2.273539,184.034302,260.320709,0.000624,...,0,0,0,0,0,0,0,0,0,0
7,tree-1,296747,469306,24882,0.184662,3.764547,0.000000,219.274033,248.468781,0.000624,...,0,0,0,0,0,0,0,0,0,0
8,jeans-1,296747,469306,24882,0.118828,14.044736,316.630463,243.695526,143.864136,0.000624,...,0,0,0,0,0,0,0,0,0,0
9,ground-1,296747,469306,24882,0.377812,199.497482,205.554993,440.502502,253.050995,0.000624,...,0,0,0,0,0,0,0,0,0,0


### Merge the old df and new df

In [14]:
import sys
sys.path.append('../')
from helper import *
top_5_match(df_old[['box_alias', 'x1','y1','w','h']], df_old.iloc[12][5:9])

[[12, 'pants-1', 1.0000000000000004],
 [8, 'jeans-1', 0.9794401995185961],
 [11, 'pavement-1', 0.3496589439291445],
 [13, 'man-2', 0.19506705558670737],
 [1, 'man-1', 0.15228061839919418]]

In [15]:
# dict list subclass (still hardcoded)
dict_list_subclass = {'person':['man', 'guy', 'boy', 'girl', 'woman'], 'man': ['boy', 'guy'], 'woman':['woman', 'girl', 'bride']}

df_old['is_old_type'] = True
df_old['is_old_attr'] = True

df_new['is_old_type'] = False
df_new['is_old_attr'] = False

    
# get all class name in old data and save the index
df_merged = df_old.copy()

dict_class_list_index = defaultdict(list)
for i, row in df_old[['box_alias']].iterrows():
    class_name = row['box_alias'][:row['box_alias'].find('-')]
    dict_class_list_index[class_name].append(i)
print('dict_class_list_index from old df: ', dict_class_list_index)

list_idx_to_be_dropped = []
for i, row in df_new.iterrows():
    print(len(df_old))
    class_name = row['box_alias'][:row['box_alias'].find('-')]
    
    if (class_name in dict_class_list_index):
        list_idx_to_be_dropped += dict_class_list_index[class_name]
        
    # handle sub-class (still hardcoded)
    if (class_name in dict_list_subclass):
        for subclass_name in dict_list_subclass[class_name]:
            list_idx_to_be_dropped += dict_class_list_index[subclass_name]
        
        
    # handle overlap boxes (choose attribute from old data instead)
    # find overlap
    result_top_match = None
    result_top_5_match = top_5_match(df_old[['box_alias', 'x1','y1','w','h']], row[5:9])
    for res in result_top_5_match:
        idx_old = res[0]
        name = res[1]
        similarity = res[2]
        if (name.split('-')[0]==class_name or name.split('-')[0]==dict_list_subclass[class_name]):
            result_top_match = (idx_old, name, similarity)
            break
    if (result_top_match != None):
        if (similarity > .6):
#             print(result_top_match)
            overlap_obj_idx = result_top_match[0]
    
            # use attribute from old data
            for col in df_old.columns:
                if ('ATTR' in col):
                    row[col] = df_old.loc[overlap_obj_idx][col]
            row['is_old_attr'] = True

    
    df_merged = df_merged.append(row, ignore_index = True)
        
print(list_idx_to_be_dropped)
set_idx_to_be_dropped = set(list_idx_to_be_dropped)   #just removing some duplicates
print(set_idx_to_be_dropped)

df_merged = df_merged.drop(set_idx_to_be_dropped).reset_index(drop=True)

dict_class_list_index from old df:  defaultdict(<class 'list'>, {'trunk': [0], 'man': [1, 13], 'elephant': [2], 'hair': [3, 10], 'shirt': [4], 'head': [5], 'face': [6], 'tree': [7], 'jeans': [8], 'ground': [9], 'pavement': [11], 'pants': [12]})
14
14
[1, 13, 2]
{1, 2, 13}


In [16]:
df_merged

,box_alias,image_id,ann_id,ref_id,salience,x1,y1,w,h,TYPE_yolk,...,TYPE_wine glass,TYPE_hot dog,TYPE_potted plant,TYPE_dining table,TYPE_tv,TYPE_cell phone,TYPE_teddy bear,TYPE_hair drier,is_old_type,is_old_attr
0,trunk-1,296747,469306,24882,0.058165,411.079834,256.495056,85.829346,199.942230,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True,True
1,hair-1,296747,469306,24882,0.021163,186.153610,83.822449,77.811691,80.242737,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True,True
2,shirt-1,296747,469306,24882,0.108199,124.160843,161.846359,173.210648,184.301651,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True,True
3,head-1,296747,469306,24882,0.182997,409.485199,0.000000,229.364655,235.395401,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True,True
4,face-1,296747,469306,24882,0.162378,455.965698,2.273539,184.034302,260.320709,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True,True
5,tree-1,296747,469306,24882,0.184662,3.764547,0.000000,219.274033,248.468781,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True,True
6,jeans-1,296747,469306,24882,0.118828,14.044736,316.630463,243.695526,143.864136,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True,True
7,ground-1,296747,469306,24882,0.377812,199.497482,205.554993,440.502502,253.050995,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True,True
8,hair-2,296747,469306,24882,0.112146,358.808258,0.000000,276.049286,119.861534,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True,True
9,pavement-1,296747,469306,24882,0.345995,0.000000,233.311859,448.940460,227.385376,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True,True


In [17]:
df_merged['ATTR_gray']

0     0.002682
1     0.002682
2     0.002666
3     0.003736
4     0.003554
5     0.002528
6     0.002905
7     0.003709
8     0.002788
9     0.003743
10    0.002905
11    0.002511
12    0.003942
Name: ATTR_gray, dtype: float64

In [18]:
df_old.loc[1][col]

True

In [19]:
is_nan = False
for col in df_merged.columns:
    is_nan = df_merged[col].isnull().values.any()
print(is_nan)

False


In [20]:
df_merged.to_csv('sample_merged.tsv', sep = '\t', index=False)

In [21]:
df_merged_from_file = pd.read_csv('sample_merged.tsv', encoding='utf-8',sep='\t')
df_merged_from_file['ATTR_blue']

0     0.002491
1     0.002500
2     0.002524
3     0.002497
4     0.002495
5     0.002496
6     0.005222
7     0.002497
8     0.002506
9     0.002534
10    0.005222
11    0.002507
12    0.002494
Name: ATTR_blue, dtype: float64

## Wrap all the process into a function

In [22]:
merged_data_path = '/scratch2/thdaryan/data/new_merged_result_using_attr_old_data'

def merge_old_and_new_df(file_idx):
    # dict list subclass (still hardcoded)
    dict_list_subclass = {'person':['man', 'guy', 'boy', 'girl', 'woman'], 'man': ['boy', 'guy'], 'woman':['woman', 'girl', 'bride']}
    
    df_old = pd.read_csv(os.path.join(old_data_path,f'refCOCO/test/attr_tables_with_target_box/attr_{file_idx}.tsv'), encoding='utf-8',sep='\t')
    df_new = pd.read_csv(os.path.join(new_data_path,f'refCOCO/test/attr_tables_with_target_box/attr_{file_idx}.tsv'), encoding='utf-8',sep='\t')
    
    # Preprocess
    
    # drop type none in new data
    list_col = df_new.columns
    for col in list_col:
        if(len(col) >= len('TYPE_None') and col[:len('TYPE_None')]=='TYPE_None'):
            df_new.drop([col], axis=1, inplace = True)
    
    list_type_old_data = [col for col in df_old.columns if 'TYPE_' in col]
    list_type_new_data = [col for col in df_new.columns if 'TYPE_' in col]
    
    # add flag to identify it is the old data or new data
    df_old['is_old_type'] = True
    df_old['is_old_attr'] = True

    df_new['is_old_type'] = False
    df_new['is_old_attr'] = False

    
    # Make the columns in old df and new df same
    for col in list_type_old_data:
        if (col not in list_type_new_data):
            df_new[col] = 0

    for col in list_type_new_data:
        if (col not in list_type_old_data):
            df_old[col] = 0
            
    # Merge old df and new df
    # get all class name in old data and save the index
    df_merged = df_old.copy()

    dict_class_list_index = defaultdict(list)
    for i, row in df_old[['box_alias']].iterrows():
        class_name = row['box_alias'][:row['box_alias'].find('-')]
        dict_class_list_index[class_name].append(i)

    list_idx_to_be_dropped = []
    for i, row in df_new.iterrows():
        class_name = row['box_alias'][:row['box_alias'].find('-')]

        if (class_name in dict_class_list_index):
            list_idx_to_be_dropped += dict_class_list_index[class_name]

        # handle sub-class (still hardcoded)
        if (class_name in dict_list_subclass):
            for subclass_name in dict_list_subclass[class_name]:
                list_idx_to_be_dropped += dict_class_list_index[subclass_name]


        # handle overlap boxes (choose attribute from old data instead)
        # find overlap
        result_top_match = None
        result_top_5_match = top_5_match(df_old[['box_alias', 'x1','y1','w','h']], row[5:9])
#         print(result_top_5_match)
        for res in result_top_5_match:
            idx_old = res[0]
            name = res[1]
            similarity = res[2]
            if (name.split('-')[0]==class_name or (class_name in dict_list_subclass and name.split('-')[0]==dict_list_subclass[class_name])):
                result_top_match = (idx_old, name, similarity)
                break
                
        if (result_top_match != None):
            if (similarity > .6):
                overlap_obj_idx = result_top_match[0]

                # use attribute from old data
                for col in df_old.columns:
                    if ('ATTR' in col):
                        row[col] = df_old.loc[overlap_obj_idx][col]
                row['is_old_attr'] = True
        
        df_merged = df_merged.append(row, ignore_index = True)
       

    set_idx_to_be_dropped = set(list_idx_to_be_dropped)   #just removing some duplicates

    df_merged = df_merged.drop(set_idx_to_be_dropped).reset_index(drop=True)
    
    # check if there is NaN value (if so then there is some error on the process)
    is_nan = False
    for col in df_merged.columns:
        is_nan = df_merged[col].isnull().values.any()
    if (is_nan):
        raise Exception('there is NaN value')
        
    # save the result
    df_merged.to_csv(os.path.join(merged_data_path, f'refCOCO/test/attr_tables_with_target_box/attr_{file_idx}.tsv'), sep = '\t', index=False)
    
    

In [23]:
merge_old_and_new_df(0)

### Process all files

In [170]:
for i in range(5000):
    merge_old_and_new_df(i)
    if (i % 50 == 0 ):
        print(f'finished processing {i} files')

finished processing 0 files
finished processing 50 files
finished processing 100 files
finished processing 150 files
finished processing 200 files
finished processing 250 files
finished processing 300 files
finished processing 350 files
finished processing 400 files
finished processing 450 files
finished processing 500 files
finished processing 550 files
finished processing 600 files
finished processing 650 files
finished processing 700 files
finished processing 750 files
finished processing 800 files
finished processing 850 files
finished processing 900 files
finished processing 950 files
finished processing 1000 files
finished processing 1050 files
finished processing 1100 files
finished processing 1150 files
finished processing 1200 files
finished processing 1250 files
finished processing 1300 files
finished processing 1350 files
finished processing 1400 files
finished processing 1450 files
finished processing 1500 files
finished processing 1550 files
finished processing 1600 files


In [25]:
!python merge_data_considering_old_attr.py

old_data_path:  /scratch2/hle
new_data_path:  /scratch2/thdaryan/data/fine_tuned
merged_data_path:  /scratch2/thdaryan/data/new_merged_result_using_attr_old_data
finished processing 0 files
^C
Traceback (most recent call last):
  File "merge_data_considering_old_attr.py", line 119, in <module>
    merge_old_and_new_df(i)
  File "merge_data_considering_old_attr.py", line 50, in merge_old_and_new_df
    df_new[col] = 0
  File "/scratch2/hle/py3_env/lib/python3.7/site-packages/pandas/core/frame.py", line 3360, in __setitem__
    indexer = convert_to_index_sliceable(self, key)
  File "/scratch2/hle/py3_env/lib/python3.7/site-packages/pandas/core/indexing.py", line 2461, in convert_to_index_sliceable
    if obj._data.items.contains(key):
  File "/scratch2/hle/py3_env/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 3930, in contains
    return key in self._engine
KeyboardInterrupt
